# NPS adjustments

Before trainning the model, there are a few caveats about the NPS and the touchpoints' data. Particularly, there are 3 adjustments we have to consider:
1) Monthly statistical adjustment (weights): Each month, the NPS scores per cabin/haul are adjusted to fit a true sample of the global distribution of cabin/haul users. That is, for the NPS per cabin haul to be representative, it is first adjusted so that the sample of users from which it is computed is representative as well. That way, if by chance a month has only one passenger in, for example, Bussines-Long Haul, the associated NPS considers that that particular sample is not representtive of the normal distribution of Business Long Haul users. Unfortunatelly, this values are computed every month, and we wont have access to them at prediction time. 
2) Change of survey: On early december 2019 and on late december 2023 there have been changes on the survey handled to the clients that have considerelly affected the NPS score and the satisfaction per touchpoint. A fixed value (per cabin/haul and globally) is used to adjust 2019 and 2022 (plus december 2019). Those values are computed externally.
3) C/H ponderation: On the global NPS computation, each cabin haul NPS is adjusted by a factor based on the percentaje of users that will fly on that particular cabine/haul. These values are fixed and at the moment of prediction we will have access to them.

## Installs and imports

In [2]:
!pip install plotly
!pip install boto3==1.19.12
!pip install s3fs

  Using cached boto3-1.19.12-py3-none-any.whl (131 kB)
  Using cached botocore-1.22.12-py3-none-any.whl (8.1 MB)
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.16-py2.py3-none-any.whl.metadata (48 kB)
Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.42
    Uninstalling botocore-1.31.42:
      Successfully uninstalled botocore-1.31.42
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.6.0
    Uninstalling s3transfer-0.6.0:
      Successfully uninstalled s3transfer

In [3]:
# General
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
import xlsxwriter
import datetime
import boto3


# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# Plots
import matplotlib.pyplot as plt
import seaborn as sns

#Warnings
import warnings
warnings.filterwarnings("ignore")

## Read files

In [4]:
nps_path = 's3://iberia-data-lake/customer/nps_surveys/export_historic/insert_date_ci=2023-09-14/'
iberia_kpis_path = 's3://iberia-data-lake/customer/one_shot/iberia_kpis_weekly/'

In [5]:
import boto3
import pandas as pd

def read_csv_files_from_s3(s3_path):
    # Initialize Boto3 client for S3
    s3_client = boto3.client('s3')

    # Extract bucket name and prefix from the S3 path
    bucket, prefix = s3_path.replace('s3://', '').split('/', 1)

    # Get a list of all keys (object paths) under the specified S3 path
    s3_resource = boto3.resource('s3')
    s3_keys = [item.key for item in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix)]

    # Generate a list of full S3 paths for all CSV files
    preprocess_paths = [f"s3://{bucket}/{key}" for key in s3_keys]

    # Read all CSV files into a list of DataFrames
    dfs = [pd.read_csv(s3_client.get_object(Bucket=bucket, Key=key)['Body']) for key in s3_keys]

    # Concatenate DataFrames into a single DataFrame
    df_concatenated = pd.concat(dfs, axis=0, ignore_index=True)

    return df_concatenated

In [6]:
# Readd files from paths
df_nps = read_csv_files_from_s3(nps_path)

In [7]:
df_nps[df_nps['monthly_weight'].notna()]['date_flight_local'].max()

'2023-08-31'

In [8]:
df_kpis = read_csv_files_from_s3(iberia_kpis_path)

In [9]:
df_kpis[df_kpis['year']==2023]


,year,month,week,haul,cabin,total_bookings,physical_compartment_capacity,load_factor,n_flights,mean_price,deviation_price,otp3_landing,otp14_landing,otp15_landing,otp60_landing,otp3_takeoff,otp14_takeoff,otp15_takeoff,otp60_takeoff
44,2023,7,26,SH,Economy,72348,76989,93.972,547,121.090,97.876,69.498,85.199,85.714,96.268,64.736,82.754,86.100,95.753
45,2023,1,1,LH,Economy,90742,96771,93.770,354,587.670,362.958,72.034,84.181,85.593,99.153,65.254,83.333,88.701,98.305
67,2023,6,26,SH,Premium,18373,21336,86.113,1484,253.690,195.659,65.359,82.122,83.285,95.833,61.531,79.167,83.140,95.930
100,2023,6,22,LH,Economy,54524,57480,94.857,210,505.910,316.543,73.810,86.190,86.190,96.667,61.429,83.810,86.667,96.667
104,2023,2,6,SH,Economy,259696,286428,90.667,2002,78.200,68.482,77.847,91.146,92.065,98.384,75.533,88.795,92.138,98.237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,2023,8,31,LH,Economy,90904,95466,95.221,350,577.330,345.432,80.899,94.101,94.663,100.000,61.236,85.955,91.573,99.719
1451,2023,7,27,LH,Economy,100597,108024,93.125,396,571.820,308.863,61.443,72.637,73.881,95.025,49.502,74.129,82.338,95.274
1452,2023,2,7,SH,Economy,267697,289121,92.590,2016,82.390,74.925,73.707,89.184,89.876,97.924,72.068,87.291,91.078,97.924
1453,2023,1,4,LH,Economy,81615,95878,85.124,350,418.130,282.986,65.254,79.379,79.944,95.198,60.169,75.989,79.379,92.090


## 1) Monthly statistical adjustment.

In [10]:
condition_1 = (df_nps['operating_airline_code'].isin(['IB', 'YW']))
condition_2 = ((df_nps['invitegroup_ib'] != 3) | (df_nps['invitegroup_ib'].isnull()))
condition_3 = (df_nps['invitegroup'] == 2)

df_nps_tkt = df_nps.loc[condition_1 & (condition_2 & condition_3)]

In [11]:
df_nps

,respondent_id,sample_id,surveyed_flight_number,date_flight_local,scheduled_departure_time_local,scheduled_arrival_time_local,tier_level,language_code,aircraft_registration_number,seat_no,volume_of_bags,number_of_child_in_the_booking,number_of_infant_in_the_booking,number_of_people_in_the_booking,infinita_customers_identifer,flag_of_ib_singular_customers,country_code,list_of_options_for_booking_channel,list_of_options_for_checkin_channel,lounge_used_at_origin_airport,customer_journey_origin,customer_journey_destination,number_of_flights_in_journey,order_of_flight_in_journey,fleet_in_surveyed_flight,marketing_airline_code,date_of_flight_gmt,scheduled_departure_time_gmt,real_departure_time_local,real_departure_time_gmt,scheduled_arrival_time_gmt,real_arrival_time_local,real_arrival_time_gmt,segment,route,overall_haul,purser,invitegroup_ib,weight_category,weekly_weight,monthly_weight,pnr_show,ff_number,id_golden_record,ticket_num,started,time_spent_hrminsec,customer_email_show,origin_of_surveyed_flight,destination_of_surveyed_flight,operating_airline_code,cabin_in_surveyed_flight,haul,first_name_show,last_name_show,second_last_name_show,nps_category,nps_100,survey_type,invitegroup,group_age_survey,pun_100_punctuality,inm_200_issues_prior_checkin,inm_200_issues_prior_ticket_change,inm_200_issues_prior_schedule_change,inm_200_issues_prior_contact_center,inm_200_issues_prior_special_serv,inm_200_issues_prior_special_req,inm_200_issues_prior_avios,inm_200_issues_prior_voucher,inm_206_issues_checkin_long_queues,inm_206_issues_checkin_wrong_info,inm_206_issues_checkin_additional_fees,inm_206_issues_checkin_overbooking,inm_206_issues_checkin_downgrade,inm_206_issues_checkin_staff,inm_206_issues_checkin_social_distance,inm_206_issues_checkin_face_masks,inm_206_issues_checkin_documentation,inm_206_issues_checkin_other,inm_207_issues_lounge_denied,inm_207_issues_lounge_overcrowded,inm_207_issues_lounge_cleanliness,inm_207_issues_lounge_wifi,inm_207_issues_lounge_staff,inm_207_issues_lounge_food_drink,inm_207_issues_lounge_face_masks,inm_207_issues_lounge_other,inm_208_issues_security_leave_sth,inm_208_issues_security_long_queues,inm_208_issues_security_staff,inm_208_issues_security_social_distance,inm_208_issues_security_face_masks,inm_208_issues_security_other,inm_209_issues_boarding_unclear,inm_209_issues_boarding_gate_changed,inm_209_issues_boarding_lack_space,inm_209_issues_boarding_disorganised,inm_209_issues_boarding_staff,inm_209_issues_boarding_social_distance,inm_209_issues_boarding_face_masks,inm_209_issues_boarding_documentation,inm_209_issues_boarding_other,inm_220_issues_timing_cancelled,inm_220_issues_timing_dep_delay,inm_220_issues_timing_arr_delay,inm_220_issues_timing_missed,inm_230_issues_onboard_staff,inm_230_issues_onboard_ife,inm_230_issues_onboard_overcrowding,inm_230_issues_onboard_face_masks,inm_235_issues_onboard_comfort_damaged,inm_235_issues_onboard_comfort_space,inm_235_issues_onboard_comfort_temperature,inm_235_issues_onboard_comfort_cleanliness,inm_235_issues_onboard_comfort_washrooms,inm_235_issues_onboard_comfort_other,inm_236_issues_meal_availability,inm_236_issues_meal_portions,inm_236_issues_meal_quality,inm_236_issues_meal_special,inm_236_issues_meal_other,inm_240_issues_baggage_lost,inm_240_issues_baggage_delayed,inm_240_issues_baggage_demaged,inm_240_issues_baggage_staff,inm_240_issues_baggage_hand,inm_240_issues_baggage_other,inm_250_issues_arrival_slow,inm_250_issues_arrival_unclear,inm_250_issues_arrival_aditional_request,inm_250_issues_arrival_staff,inm_250_issues_arrival_immigration_queues,inm_250_issues_arrival_immigration_passport,inm_250_issues_arrival_immigration_other,inm_255_issues_connecting_missed,inm_255_issues_connecting_staff,inm_255_issues_connecting_baggage,inm_255_issues_connecting_other,bkg_100_booking,bkg_200_journey_preparation,inm_400_issues_response,pfl_100_checkin,pfl_200_security,pfl_300_lounge,pfl_500_boarding,ifl_100_cabin_crew,ifl_200_flight_crew_annoucements,ifl_300_cabin,

In [12]:
datetime_features = ['date_flight_local', 'scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local',
                     'real_arrival_time_local', 'started']
columns_to_cross_kpis=['cabin_in_surveyed_flight','haul']
columns_ext = ['tier_level', 'language_code', 'seat_no', 'volume_of_bags', 'number_of_child_in_the_booking', 'number_of_infant_in_the_booking',
              'number_of_people_in_the_booking', 'country_code', 'customer_journey_origin', 'customer_journey_destination', 'number_of_flights_in_journey',
              'order_of_flight_in_journey', 'marketing_airline_code', 'overall_haul', 'weight_category', 'ff_number', 'ticket_num', 'operating_airline_code',
               'nps_category', 'nps_100', 'group_age_survey', 'gender'] # invite_group

#'bkg_100_booking', 
touchpoints = ['bkg_200_journey_preparation', 'pfl_100_checkin', 'pfl_200_security', 'pfl_300_lounge',
               'pfl_500_boarding', 'ifl_300_cabin', 'ifl_200_flight_crew_annoucements', 'ifl_600_wifi', 'ifl_500_ife',
               'ifl_400_food_drink', 'ifl_100_cabin_crew', 'arr_100_arrivals', 'con_100_connections', 'pun_100_punctuality',
               'loy_200_loyalty_programme', 'inm_400_issues_response', 'img_310_ease_contact_phone']
survey_fields = ['cla_600_wifi_t_f', 'tvl_journey_reason']

for feat in datetime_features:
    if feat in ['scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local', 'real_arrival_time_local','date_flight_local']:
        df_nps_tkt[feat] = pd.to_datetime(df_nps_tkt[feat], format="%Y%m%d %H:%M:%S", errors = 'coerce')
    else:
        df_nps_tkt[feat] = pd.to_datetime(df_nps_tkt[feat], errors = 'ignore')
df_nps_tkt['time_spent_hrminsec'] = pd.to_timedelta(df_nps_tkt['time_spent_hrminsec']).dt.total_seconds()
df_nps_tkt['started_hour'] = df_nps_tkt['started'].dt.hour
df_nps_tkt['year_flight'] = df_nps_tkt['date_flight_local'].dt.year
df_nps_tkt['month_flight'] = df_nps_tkt['date_flight_local'].dt.month
df_nps_tkt['day_flight'] = df_nps_tkt['date_flight_local'].dt.day
df_nps_tkt['weekday_flight'] = df_nps_tkt['date_flight_local'].dt.weekday
df_nps_tkt['is_weekend_or_friday_flight'] = df_nps_tkt['weekday_flight'].apply(lambda x: 1 if x in [5, 6,7] else 0)
df_nps_tkt['delay_departure'] = (df_nps_tkt['real_departure_time_local'] - df_nps_tkt['scheduled_departure_time_local']).dt.total_seconds()/60
df_nps_tkt['delay_arrival'] = (df_nps_tkt['real_arrival_time_local'] - df_nps_tkt['scheduled_arrival_time_local']).dt.total_seconds()/60
datetime_features = datetime_features + ['time_spent_hrminsec', 'started_hour', 'year_flight', 'month_flight',
                                         'day_flight', 'weekday_flight', 'is_weekend_or_friday_flight']

In [13]:
df_nps_tkt=df_nps_tkt[df_nps_tkt['date_flight_local'].dt.year >= 2019]

In [14]:
df_nps_tkt['date_flight_local'].min()

Timestamp('2019-01-01 00:00:00')

In [15]:
columns_to_select = datetime_features + columns_ext + touchpoints + ['delay_arrival', 'delay_departure', 'ticket_price']+columns_to_cross_kpis+['monthly_weight']

In [16]:
df_nps_tkt = df_nps_tkt[columns_to_select]
df_nps_tkt[['date_flight_local','nps_100','monthly_weight']]

,date_flight_local,nps_100,monthly_weight
6,2023-01-04,8,0.519
7,2023-01-20,10,0.519
9,2023-01-01,10,0.519
10,2023-01-08,10,0.519
11,2023-01-29,10,0.414
...,...,...,...
854016,2023-07-19,0,0.565
854017,2023-07-28,9,0.565
854018,2023-06-21,9,0.469
854019,2023-08-10,9,0.545


In [17]:
x=df_nps_tkt.copy()
pd.set_option('display.max_rows', 100)
x = x[(x['date_flight_local']=='2022-12-01') & (x['cabin_in_surveyed_flight'] == 'Economy') & (x['haul'] == 'MH') ]
display(x[(x['nps_100'] > 8) | (x['nps_100'] < 6)][['date_flight_local','cabin_in_surveyed_flight','haul','monthly_weight','nps_100']].head(200))



,date_flight_local,cabin_in_surveyed_flight,haul,monthly_weight,nps_100
6915,2022-12-01,Economy,MH,2.708,10
19939,2022-12-01,Economy,MH,2.708,9
37137,2022-12-01,Economy,MH,2.708,5
38759,2022-12-01,Economy,MH,2.708,9
39319,2022-12-01,Economy,MH,2.708,3
39618,2022-12-01,Economy,MH,2.708,10
40091,2022-12-01,Economy,MH,2.708,9
40092,2022-12-01,Economy,MH,2.708,5
73083,2022-12-01,Economy,MH,2.708,10
74319,2022-12-01,Economy,MH,2.708,10


#### If 4 cabin/haul

Before aggregating, I need to rewrite come categories in order to comapre with the dashboard:

In [18]:
df_nps_tkt['haul'] = df_nps_tkt['haul'].replace('MH', 'SH')
#df_nps_tkt['cabin_in_surveyed_flight'] = df_nps_tkt['cabin_in_surveyed_flight'].replace('Premium Economy', 'Economy')

#df_kpis['cabin']=df_kpis['cabin'].replace('Premium Economy','Premium Economy')
df_kpis['cabin']=df_kpis['cabin'].replace('Premium','Business')


In [19]:
df_nps_tkt[df_nps_tkt['monthly_weight'].isnull()][['date_flight_local','cabin_in_surveyed_flight','haul']]

,date_flight_local,cabin_in_surveyed_flight,haul
32457,2023-09-09,Economy,LH
32462,2023-09-09,Economy,SH
32465,2023-09-07,Economy,LH
32483,2023-09-10,Business,LH
32484,2023-09-04,Economy,SH
...,...,...,...
853982,2023-09-04,Economy,SH
853990,2023-09-10,Economy,SH
854006,2023-09-09,Economy,SH
854007,2023-09-06,Economy,LH


In [20]:
df_nps_tkt = df_nps_tkt[df_nps_tkt['monthly_weight'].notnull()]

### Aggregate on time frequency

In [21]:
# Function to calculate NPS
def calculate_nps(promoters, detractors, passives):
    total_responses = promoters + detractors + passives
    nps = 100 * (promoters - detractors) / total_responses
    return nps

def calculate_aggregated_features_with_cross_kpis(df, variables, columns_to_cross_kpis, time_frequency='M', calculate_satisfaction=True):
    touchpoints = [var for var in variables if calculate_satisfaction]
    non_touchpoints = [var for var in variables if not calculate_satisfaction]

    # Combine the values of columns_to_cross_kpis into a new column 'cross_kpis'
    df['cross_kpis'] = df[columns_to_cross_kpis].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)

    # Group the data by 'cross_kpis' and 'date_flight_local' using pd.Grouper on the entire DataFrame
    grouped_data = df.groupby([pd.Grouper(key='cross_kpis'), pd.Grouper(key='date_flight_local', freq=time_frequency)], group_keys=False)

    # Calculate the aggregated features for each group
    aggregated_features_data = pd.DataFrame()
    
    # Calculate the aggregated features for each group
    aggregated_features_data['NPS'] = grouped_data.apply(lambda x: calculate_nps(
        sum((x['nps_100'] == 9) | (x['nps_100'] == 10)),
        sum(x['nps_100'] <= 6),
        sum((x['nps_100'] == 7) | (x['nps_100'] == 8))
    ))

    # Calculate promoters_weight and detractors_weight
    aggregated_features_data['promoters_weight'] = grouped_data.apply(lambda x: (x[x['nps_100'] > 8]['monthly_weight'].sum()) if (x['nps_100'] > 8).any() else 0)
    aggregated_features_data['detractors_weight'] = grouped_data.apply(lambda x: (x[x['nps_100'] <= 6]['monthly_weight'].sum()) if (x['nps_100'] <= 6).any() else 0)




    # Calculate monthly_weight within each group
    aggregated_features_data['monthly_weight'] = grouped_data.apply(lambda x: x['monthly_weight'].sum())

    # Calculate NPS_weighted using the same logic as your SQL query
    aggregated_features_data['NPS_weighted'] = (aggregated_features_data['promoters_weight'] - aggregated_features_data['detractors_weight']) * 100 / aggregated_features_data['monthly_weight']

    for col in touchpoints + non_touchpoints:
        aggregated_features_data[f'{col}_sum'] = grouped_data[col].sum()
        aggregated_features_data[f'{col}_std'] = grouped_data[col].std()
        aggregated_features_data[f'{col}_mean'] = grouped_data[col].mean()
        aggregated_features_data[f'{col}_not_nulls'] = grouped_data[col].apply(lambda x: x.notnull().sum())
        if col in touchpoints:
            aggregated_features_data[f'{col}_satisfaction'] = grouped_data[col].apply(lambda x: x[x >= 8].count() / x.count()*100)
        else:
            aggregated_features_data[f'{col}_max'] = grouped_data[col].max()
            aggregated_features_data[f'{col}_min'] = grouped_data[col].min()
    
    # Reset the index of aggregated_features_data
    aggregated_features_data.reset_index(inplace=True)

    # Split the 'cross_kpis' column back into individual columns and concatenate them to aggregated_features_data
    split_columns = aggregated_features_data['cross_kpis'].str.split('_', expand=True)
    split_columns.columns = columns_to_cross_kpis
    aggregated_features_data = pd.concat([split_columns, aggregated_features_data], axis=1)
    
    # Drop the temporary 'cross_kpis' column
    aggregated_features_data.drop(columns=['cross_kpis'], inplace=True)
    
    return aggregated_features_data


In [22]:
df=calculate_aggregated_features_with_cross_kpis(df_nps_tkt, touchpoints, columns_to_cross_kpis, time_frequency='W', calculate_satisfaction=True)

In [23]:
df.describe()

,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction
count,1208.000,1208.000,1208.000,1208.000,1208.000,1208.000,1205.000,1208.000,1208.000,1208.000,1208.000,1205.000,1208.000,1208.000,1208.000,1208.000,1205.000,1208.000,1208.000,1208.000,1208.000,1139.000,1164.000,1208.000,1164.000,1208.000,1205.000,1208.000,1208.000,1208.000,1208.000,1205.000,1208.000,1208.000,1208.000,1208.000,1204.000,1208.000,1208.000,1208.000,1208.000,1099.000,1152.000,1208.000,1152.000,1208.000,1155.000,1173.000,1208.000,1173.000,1208.000,1201.000,1206.000,1208.000,1206.000,1208.000,1205.000,1208.000,1208.000,1208.000,1208.000,1204.000,1208.000,1208.000,1208.000,1208.000,1157.000,1178.000,1208.000,1178.000,1208.000,1205.000,1208.000,1208.000,1208.000,1208.000,1196.000,1203.000,1208.000,1203.000,1208.000,1129.000,1173.000,1208.000,1173.000,1208.000,1201.000,1206.000,1208.000,1206.000
mean,31.501,221.734,96.223,416.451,31.474,2773.946,2.703,7.541,370.656,65.828,3312.102,2.385,7.962,417.863,72.756,3424.096,2.043,8.222,415.526,75.930,385.517,2.235,7.857,48.599,69.632,3318.470,2.271,7.869,422.544,69.894,3352.906,2.132,8.016,423.516,71.934,3430.777,1.977,8.321,414.496,76.537,413.988,3.423,4.912,77.473,30.718,871.149,2.458,7.396,112.674,62.858,1868.399,2.707,6.744,278.323,50.764,3448.689,2.231,8.172,416.824,74.973,3323.021,2.175,7.949,416.498,71.089,887.709,2.675,7.369,120.981,63.684,3532.512,2.315,8.530,428.362,81.952,1438.302,2.237,7.412,196.521,58.233,161.859,3.208,3.249,54.925,14.982,1290.034,3.123,6.157,213.206,43.977
std,17.354,338.425,148.434,632.531,17.380,4374.095,0.469,0.571,579.858,9.270,5312.255,0.465,0.495,663.125,8.718,5470.022,0.372,0.355,660.501,7.099,371.824,0.675,0.845,46.411

### Merge dfs

In [24]:
# Convert the 'date_flight_local' column in result_df to separate 'year' and 'month' columns
df['year'] = df['date_flight_local'].dt.year
df['month'] = df['date_flight_local'].dt.month
df['week'] = df['date_flight_local'].dt.week
# Perform the merge based on the specified columns
merged_df = df.merge(df_kpis, 
                            left_on=['year', 'month', 'week','cabin_in_surveyed_flight', 'haul'], 
                            right_on=['year', 'month','week', 'cabin', 'haul'], 
                            how='inner')

# Drop the redundant columns from the merged dataframe
merged_df.drop(columns=['cabin_in_surveyed_flight'], inplace=True)

# Reorder the columns to place 'year', 'month', 'cabin', and 'haul' at the beginning
cols_to_move = ['date_flight_local','year', 'month', 'week', 'cabin', 'haul']
merged_df = merged_df[cols_to_move + [col for col in merged_df.columns if col not in cols_to_move]]
merged_df.head()

,date_flight_local,year,month,week,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,total_bookings,physical_compartment_capacity,load_factor,n_flights,mean_price,deviation_price,otp3_landing,otp14_landing,otp15_landing,otp60_landing,otp3_takeoff,otp14_takeoff,otp15_takeoff,otp60_takeoff
0,2019-01-06,2019,1,1,Business,LH,42.500,12.780,4.090,20.448,42.500,288.000,2.378,8.000,36,69.444,322.000,2.207,8.050,40,72.500,307.000,1.682,8.528,36,83.333,228.000,2.685,7.125,32,62.500,333.000,1.859,8.325,40,70.000,316.000,2.373,7.900,40,72.500,316.000,2.113,8.103,39,76.923,102.000,3.102,6.000,17,35.294,259.000,2.379,7.400,35,57.143,296.000,2.325,7.590,39,58.974,326.000,2.304,8.150,40,75.000,304.000,1.786,8.000,38,65.789,35.000,1.258,8.750,4,75.000,366.000,1.210,9.150,40,90.000,309.000,2.112,7.725,40,62.500,4.000,0.000,2.000,2,0.000,227.000,2.266,6.676,34,41.176,6809,8695,78.309,296,913.220,855.953,56.913,78.135,79.421,96.141,59.164,81.029,82.958,95.820
1,2019-01-13,2019,1,2,Business,LH,58.140,30.161,4.601,43.963,58.140,578.000,1.824,8.500,68,79.412,741.000,1.829,8.616,86,86.047,714.000,1.912,8.602,83,85.542,498.000,1.695,8.164,61,77.049,708.000,1.870,8.233,86,79.070,735.000,1.599,8.547,86,77.907,731.000,1.612,8.600,85,81.176,295.000,3.248,6.146,48,43.750,596.000,1.465,8.278,72,72.222,685.000,1.949,7.965,86,66.279,726.000,1.780,8.442,86,76.744,719.000,2.103,8.360,86,80.233,60.000,1.134,8.571,7,85.714,760.000,1.833,8.837,86,84.884,698.000,1.811,8.116,86,74.419,53.000,3.268,5.300,10,30.000,553.000,2.420,7.373,75,60.000,7879,10220,77.094,348,985.650,893.712,56.906,77.901,80.110,98.343,65.193,83.149,85.912,98.066
2,2019-01-20,2019,1,3,Business,LH,42.056,30.161,7.15

In [25]:
merged_df.describe()

,year,month,week,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,total_bookings,physical_compartment_capacity,load_factor,n_flights,mean_price,deviation_price,otp3_landing,otp14_landing,otp15_landing,otp60_landing,otp3_takeoff,otp14_takeoff,otp15_takeoff,otp60_takeoff
count,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1139.000,1164.000,1206.000,1164.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1203.000,1206.000,1206.000,1206.000,1206.000,1099.000,1150.000,1206.000,1150.000,1206.000,1154.000,1171.000,1206.000,1171.000,1206.000,1200.000,1204.000,1206.000,1204.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1157.000,1177.000,1206.000,1177.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1196.000,1203.000,1206.000,1203.000,1206.000,1128.000,1172.000,1206.000,1172.000,1206.000,1201.000,1205.000,1206.000,1205.000,1206.000,1206.000,1206.000,1206.000,1201.000,1201.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000
mean,2020.879,6.232,25.455,31.553,222.101,96.381,417.140,31.526,2778.533,2.704,7.543,371.268,65.854,3317.578,2.387,7.963,418.553,72.793,3429.755,2.043,8.222,416.213,75.932,386.156,2.235,7.857,48.680,69.632,3323.958,2.270,7.870,423.242,69.885,3358.449,2.132,8.016,424.216,71.970,3436.450,1.979,8.323,415.181,76.581,414.668,3.423,4.914,77.600,30.772,872.577,2.456,7.396,112.858,62.837,1871.484,2.705,6.746,278.782,50.807,3454.390,2.229,8.173,417.51

In [26]:
merged_df['date_flight_local'].max()

Timestamp('2023-09-03 00:00:00')

## 2) Change of survey adjustment.

Since the adjustment is done separatelly in the global NPS and in the Cabin/haul NPS we need to have a separate dataframe for the NPS.  

In [27]:
# Sample adjustments DataFrame
adjustments_data = {'haul': ['LH', 'LH', 'LH', 'SH', 'SH', 'SH'],
                    'cabin': ['Economy', 'Premium Economy' , 'Business', 'Economy', 'Premium Economy', 'Business'],
                    'adjustment_2019': [-0.55475719, -0.55475719, -6.86337594, -5.25927601, -5.25927601, -5.64891133],
                    'adjustment_2022': [1.41878561, 1.41878561, 0.42510779, -3.71320153, -3.71320153, -4.10337349]}

adjustments_df = pd.DataFrame(adjustments_data)
df=merged_df.copy()

for index, row in adjustments_df.iterrows():
    condition = ((df['haul'] == row['haul']) | (df['haul'] == 'MH')) & (df['cabin'] == row['cabin'])
    
    mask = (df['date_flight_local'].dt.year == 2019) & (df['date_flight_local'].dt.month != 12)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2019']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2019']
    
    mask = (df['date_flight_local'].dt.year == 2019) & (df['date_flight_local'].dt.month == 12)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2022']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2022']
    
    mask = (df['date_flight_local'].dt.year == 2022)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2022']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2022']
    
df[df['date_flight_local'] == '2022-12-31'].head()


,date_flight_local,year,month,week,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,total_bookings,physical_compartment_capacity,load_factor,n_flights,mean_price,deviation_price,otp3_landing,otp14_landing,otp15_landing,otp60_landing,otp3_takeoff,otp14_takeoff,otp15_takeoff,otp60_takeoff


In [28]:
df['cabin'].unique()

array(['Business', 'Economy', 'Premium Economy'], dtype=object)

Lets check the NPS curve now against the dashboard:

In [29]:
shp_bi = pd.read_excel('inputs/SH Premium.xlsx')
shp_bi

,Quarter,Month_Name,2019,2022,2023
0,Q1,Jan,38.240,44.246,35.708
1,Q1,Feb,46.748,34.647,48.818
2,Q1,Mar,40.184,38.806,35.570
3,Q2,Apr,36.272,32.042,45.796
4,Q2,May,46.937,27.681,44.219
5,Q2,Jun,18.644,25.831,40.074
6,Q3,Jul,33.755,28.475,41.470
7,Q3,Aug,35.874,36.682,48.652
8,Q3,Sep,33.372,28.024,NaN
9,Q4,Oct,43.783,44.776,NaN


In [30]:
data=df[(df['date_flight_local'].dt.year.isin([2019, 2022, 2023])) & (df['cabin'] == 'Business') & (df['haul'] == 'SH')][['NPS', 'NPS_weighted', 'date_flight_local']]


In [31]:
# Pivot the dataframe
pivot_df = data.pivot_table(index=data['date_flight_local'].dt.month, columns=data['date_flight_local'].dt.year, values=['NPS', 'NPS_weighted'], aggfunc='mean')

# Rename the columns
pivot_df.columns = [f"{year}" for _, year in pivot_df.columns]

# Reset the index
pivot_df.reset_index(inplace=True)


pivot_df.rename(columns={'date_flight_local': 'Month_Name'}, inplace=True)

In [32]:
pivot_df

,Month_Name,2019,2022,2023,2019,2022,2023
0,1,37.770,45.107,36.377,37.770,45.315,35.106
1,2,45.579,35.714,48.774,45.479,35.727,48.779
2,3,42.006,38.647,34.609,41.560,38.738,34.551
3,4,34.974,32.518,46.205,34.974,32.601,46.231
4,5,44.242,27.366,44.694,44.240,28.079,44.694
5,6,24.697,26.184,40.386,24.891,26.223,40.352
6,7,31.430,28.145,41.693,31.430,28.075,41.704
7,8,35.108,37.932,48.322,35.247,37.932,48.250
8,9,32.330,30.144,53.023,33.870,30.296,53.023
9,10,44.540,40.677,NaN,44.404,40.129,NaN


In [33]:
df.to_csv('outputs/weekly_NPS_5ch_adjusted_complete', index=False)

#### Touchpoints' satisfaction  adjustment for the change of survey

In [34]:
df=pd.read_csv('outputs/weekly_NPS_5ch_adjusted_complete')
df.head()

,date_flight_local,year,month,week,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,total_bookings,physical_compartment_capacity,load_factor,n_flights,mean_price,deviation_price,otp3_landing,otp14_landing,otp15_landing,otp60_landing,otp3_takeoff,otp14_takeoff,otp15_takeoff,otp60_takeoff
0,2019-01-06,2019,1,1,Business,LH,35.637,12.780,4.090,20.448,35.637,288.000,2.378,8.000,36,69.444,322.000,2.207,8.050,40,72.500,307.000,1.682,8.528,36,83.333,228.000,2.685,7.125,32,62.500,333.000,1.859,8.325,40,70.000,316.000,2.373,7.900,40,72.500,316.000,2.113,8.103,39,76.923,102.000,3.102,6.000,17,35.294,259.000,2.379,7.400,35,57.143,296.000,2.325,7.590,39,58.974,326.000,2.304,8.150,40,75.000,304.000,1.786,8.000,38,65.789,35.000,1.258,8.750,4,75.000,366.000,1.210,9.150,40,90.000,309.000,2.112,7.725,40,62.500,4.000,0.000,2.000,2,0.000,227.000,2.266,6.676,34,41.176,6809,8695,78.309,296,913.220,855.953,56.913,78.135,79.421,96.141,59.164,81.029,82.958,95.820
1,2019-01-13,2019,1,2,Business,LH,51.276,30.161,4.601,43.963,51.276,578.000,1.824,8.500,68,79.412,741.000,1.829,8.616,86,86.047,714.000,1.912,8.602,83,85.542,498.000,1.695,8.164,61,77.049,708.000,1.870,8.233,86,79.070,735.000,1.599,8.547,86,77.907,731.000,1.612,8.600,85,81.176,295.000,3.248,6.146,48,43.750,596.000,1.465,8.278,72,72.222,685.000,1.949,7.965,86,66.279,726.000,1.780,8.442,86,76.744,719.000,2.103,8.360,86,80.233,60.000,1.134,8.571,7,85.714,760.000,1.833,8.837,86,84.884,698.000,1.811,8.116,86,74.419,53.000,3.268,5.300,10,30.000,553.000,2.420,7.373,75,60.000,7879,10220,77.094,348,985.650,893.712,56.906,77.901,80.110,98.343,65.193,83.149,85.912,98.066
2,2019-01-20,2019,1,3,Business,LH,35.193,30.161,7.15

In [35]:
df['date_flight_local']=pd.to_datetime(df['date_flight_local'])

In [36]:
import pandas as pd
from io import StringIO

#Flight Booking
data = """
Touchpoint
IB Plus loyalty program
Boarding
Ease of contact by phone
IFE
Connections experience
In flight food and beverage
Airport security
Wi-Fi
Lounge
Arrivals experience
Check-in
Journey preparation support
Pilot's announcements
Aircraft interior
Cabin Crew
Punctuality
Response provided to the issue
"""
#5.0
data2 = """
Global adjustment
12.7
8.2
7.0
6.9
5.7
5.6
5.2
5.2
5.1
4.5
3.3
3.2
2.5
2.4
1.1
-0.2
-3.1
"""
#3.6
data3 = """
SH_Business
16.1
4.0
0.8
1.0
3.3
4.0
1.5
10.2
1.5
3.1
-1.2
4.8
-0.7
0.5
-3.0
-2.5
-2.6
"""
#7.5
data4 = """
LH_Business
11.3
5.5
11.4
5.0
12.6
3.3
14.0
2.3
8.2
6.2
0.8
7.9
1.2
2.1
3.6
1.2
3.4
"""
#4.6
data5 = """
SH_Economy
12.7
8.1
5.2
3.4
4.2
5.6
3.8
6.5
6.7
3.8
3.2
3.0
2.3
2.7
1.5
-0.8
-3.8
"""
#4.6
data5 = """
SH_Premium Economy
12.7
8.1
5.2
3.4
4.2
5.6
3.8
6.5
6.7
3.8
3.2
3.0
2.3
2.7
1.5
-0.8
-3.8
"""
#7.4
data6 = """
LH_Economy
11.2
10.5
10.1
7.4
10.0
8.6
9.6
1.8
9.8
7.5
7.0
3.6
3.5
4.4
3.9
1.7
-1.1
"""
#7.4
data6 = """
LH_Premium Economy
11.2
10.5
10.1
7.4
10.0
8.6
9.6
1.8
9.8
7.5
7.0
3.6
3.5
4.4
3.9
1.7
-1.1
"""

# Create StringIO objects to simulate file-like input
data_stream = StringIO(data)
data2_stream = StringIO(data2)
data3_stream = StringIO(data3)
data4_stream = StringIO(data4)
data5_stream = StringIO(data5)
data6_stream = StringIO(data6)

# Create DataFrames
ta_df = pd.DataFrame({'Touchpoint': data_stream.getvalue().split('\n')[1:]})
ta_df['Global adjustment'] = data2_stream.getvalue().split('\n')[1:]
ta_df['SH_Business'] = data3_stream.getvalue().split('\n')[1:]
ta_df['LH_Business'] = data4_stream.getvalue().split('\n')[1:]
ta_df['SH_Economy'] = data5_stream.getvalue().split('\n')[1:]
ta_df['LH_Economy'] = data6_stream.getvalue().split('\n')[1:]

# Display the DataFrame
ta_df=ta_df.drop(0)
ta_df=ta_df.drop(18)
ta_df


,Touchpoint,Global adjustment,SH_Business,LH_Business,SH_Economy,LH_Economy
1,IB Plus loyalty program,12.7,16.1,11.3,12.7,11.2
2,Boarding,8.2,4.0,5.5,8.1,10.5
3,Ease of contact by phone,7.0,0.8,11.4,5.2,10.1
4,IFE,6.9,1.0,5.0,3.4,7.4
5,Connections experience,5.7,3.3,12.6,4.2,10.0
6,In flight food and beverage,5.6,4.0,3.3,5.6,8.6
7,Airport security,5.2,1.5,14.0,3.8,9.6
8,Wi-Fi,5.2,10.2,2.3,6.5,1.8
9,Lounge,5.1,1.5,8.2,6.7,9.8
10,Arrivals experience,4.5,3.1,6.2,3.8,7.5


In [37]:
satisfaction_mapping = {
    'IB Plus loyalty program': 'loy_200_loyalty_programme_satisfaction',
    'Boarding': 'pfl_500_boarding_satisfaction',
    'Ease of contact by phone': 'img_310_ease_contact_phone_satisfaction',
    'IFE': 'ifl_500_ife_satisfaction',
    'Connections experience': 'con_100_connections_satisfaction',
    'In flight food and beverage': 'ifl_400_food_drink_satisfaction',
    'Airport security': 'pfl_200_security_satisfaction',
    'Wi-Fi': 'ifl_600_wifi_satisfaction',
    'Lounge': 'pfl_300_lounge_satisfaction',
    #'Flight Booking': 'bkg_100_booking_satisfaction',
    'Arrivals experience': 'arr_100_arrivals_satisfaction',
    'Check-in': 'pfl_100_checkin_satisfaction',
    'Journey preparation support': 'bkg_200_journey_preparation_satisfaction',
    "Pilot's announcements": 'ifl_200_flight_crew_annoucements_satisfaction',
    'Aircraft interior': 'ifl_300_cabin_satisfaction',
    'Cabin Crew': 'ifl_100_cabin_crew_satisfaction',
    'Punctuality': 'pun_100_punctuality_satisfaction',
    'Response provided to the issue': 'inm_400_issues_response_satisfaction'
}

# Create a Series to use for mapping
satisfaction_series = pd.Series(satisfaction_mapping)

# Replace touchpoints with their satisfaction equivalents in the original DataFrame
ta_df['Touchpoint'] = ta_df['Touchpoint'].map(satisfaction_series)


#Flip the dataframe to match the NPS one:
flipped_df = ta_df.transpose()
flipped_df.columns = flipped_df.iloc[0]
ta_df = flipped_df.drop(flipped_df.index[0])
ta_df


Touchpoint,loy_200_loyalty_programme_satisfaction,pfl_500_boarding_satisfaction,img_310_ease_contact_phone_satisfaction,ifl_500_ife_satisfaction,con_100_connections_satisfaction,ifl_400_food_drink_satisfaction,pfl_200_security_satisfaction,ifl_600_wifi_satisfaction,pfl_300_lounge_satisfaction,arr_100_arrivals_satisfaction,pfl_100_checkin_satisfaction,bkg_200_journey_preparation_satisfaction,ifl_200_flight_crew_annoucements_satisfaction,ifl_300_cabin_satisfaction,ifl_100_cabin_crew_satisfaction,pun_100_punctuality_satisfaction,inm_400_issues_response_satisfaction
Global adjustment,12.7,8.2,7.0,6.9,5.7,5.6,5.2,5.2,5.1,4.5,3.3,3.2,2.5,2.4,1.1,-0.2,-3.1
SH_Business,16.1,4.0,0.8,1.0,3.3,4.0,1.5,10.2,1.5,3.1,-1.2,4.8,-0.7,0.5,-3.0,-2.5,-2.6
LH_Business,11.3,5.5,11.4,5.0,12.6,3.3,14.0,2.3,8.2,6.2,0.8,7.9,1.2,2.1,3.6,1.2,3.4
SH_Economy,12.7,8.1,5.2,3.4,4.2,5.6,3.8,6.5,6.7,3.8,3.2,3.0,2.3,2.7,1.5,-0.8,-3.8
LH_Economy,11.2,10.5,10.1,7.4,10.0,8.6,9.6,1.8,9.8,7.5,7.0,3.6,3.5,4.4,3.9,1.7,-1.1


In [38]:
df.describe()

,year,month,week,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,total_bookings,physical_compartment_capacity,load_factor,n_flights,mean_price,deviation_price,otp3_landing,otp14_landing,otp15_landing,otp60_landing,otp3_takeoff,otp14_takeoff,otp15_takeoff,otp60_takeoff
count,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1139.000,1164.000,1206.000,1164.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1203.000,1206.000,1206.000,1206.000,1206.000,1099.000,1150.000,1206.000,1150.000,1206.000,1154.000,1171.000,1206.000,1171.000,1206.000,1200.000,1204.000,1206.000,1204.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1157.000,1177.000,1206.000,1177.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1196.000,1203.000,1206.000,1203.000,1206.000,1128.000,1172.000,1206.000,1172.000,1206.000,1201.000,1205.000,1206.000,1205.000,1206.000,1206.000,1206.000,1206.000,1201.000,1201.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000
mean,2020.879,6.232,25.455,30.602,222.101,96.381,417.140,30.575,2778.533,2.704,7.543,371.268,65.854,3317.578,2.387,7.963,418.553,72.793,3429.755,2.043,8.222,416.213,75.932,386.156,2.235,7.857,48.680,69.632,3323.958,2.270,7.870,423.242,69.885,3358.449,2.132,8.016,424.216,71.970,3436.450,1.979,8.323,415.181,76.581,414.668,3.423,4.914,77.600,30.772,872.577,2.456,7.396,112.858,62.837,1871.484,2.705,6.746,278.782,50.807,3454.390,2.229,8.173,417.51

In [39]:
original_df=df.copy()
adjustment_df=ta_df.drop("Global adjustment")

# Convert the values in the adjustment_df to numeric
adjustment_df = adjustment_df.apply(pd.to_numeric, errors='coerce')

# Filter out the rows where the year is 2023
adjusted_years = original_df['date_flight_local'].dt.year != 2023
adjusted_df = original_df[adjusted_years].copy()

# Add the adjustments to the corresponding touchpoint columns
for index, row in adjustment_df.iterrows():
    haul, cabin = index.split('_')
    mask = (adjusted_df['haul'] == haul) & (adjusted_df['cabin'] == cabin)
    for touchpoint_col in adjustment_df.columns:
        adjusted_df[touchpoint_col].fillna(0, inplace=True) 
        adjusted_df.loc[mask, touchpoint_col] += row[touchpoint_col]
        
# Concatenate adjusted_df with values from 2023
adjusted_df = pd.concat([adjusted_df, original_df[~adjusted_years]])




In [40]:
adjusted_df.describe()

,year,month,week,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,total_bookings,physical_compartment_capacity,load_factor,n_flights,mean_price,deviation_price,otp3_landing,otp14_landing,otp15_landing,otp60_landing,otp3_takeoff,otp14_takeoff,otp15_takeoff,otp60_takeoff
count,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1139.000,1164.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1203.000,1206.000,1206.000,1206.000,1206.000,1099.000,1150.000,1206.000,1206.000,1206.000,1154.000,1171.000,1206.000,1206.000,1206.000,1200.000,1204.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1157.000,1177.000,1206.000,1206.000,1206.000,1204.000,1206.000,1206.000,1206.000,1206.000,1196.000,1203.000,1206.000,1206.000,1206.000,1128.000,1172.000,1206.000,1206.000,1206.000,1201.000,1205.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1201.000,1201.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000,1206.000
mean,2020.879,6.232,25.455,30.602,222.101,96.381,417.140,30.575,2778.533,2.704,7.543,371.268,69.154,3317.578,2.387,7.963,418.553,74.475,3429.755,2.043,8.222,416.213,80.862,386.156,2.235,7.857,48.680,71.691,3323.958,2.270,7.870,423.242,74.705,3358.449,2.132,8.016,424.216,73.632,3436.450,1.979,8.323,415.181,77.660,414.668,3.423,4.914,77.600,32.921,872.577,2.456,7.396,112.858,63.888,1871.484,2.705,6.746,278.782,54.413,3454.390,2.229,8.173,417.51

In [41]:
df_kpis[(df_kpis['year']==2019)  & (df_kpis['cabin']== 'Economy') & (df_kpis['haul']== 'SH')]

,year,month,week,haul,cabin,total_bookings,physical_compartment_capacity,load_factor,n_flights,mean_price,deviation_price,otp3_landing,otp14_landing,otp15_landing,otp60_landing,otp3_takeoff,otp14_takeoff,otp15_takeoff,otp60_takeoff
38,2019,8,35,SH,Economy,219780,244607,89.850,1820,84.310,65.016,63.396,80.181,81.442,95.902,67.573,81.915,84.988,96.533
41,2019,3,13,SH,Economy,213387,229518,92.972,1551,78.260,68.009,75.698,90.677,91.795,98.883,82.053,92.563,94.064,98.883
53,2019,10,44,SH,Economy,169941,186132,91.301,1377,84.210,71.691,82.087,92.754,93.159,98.957,80.522,93.159,94.667,98.957
99,2019,10,42,SH,Economy,285237,312327,91.326,2318,82.050,68.432,67.383,84.717,85.577,97.982,76.348,89.381,91.201,97.949
108,2019,4,16,SH,Economy,236123,268464,87.953,1988,80.070,66.771,72.143,87.821,88.929,98.357,78.071,89.679,91.071,98.357
168,2019,9,35,SH,Economy,40864,43496,93.949,326,88.130,69.411,49.188,62.181,62.413,89.559,54.292,69.142,71.926,90.719
233,2019,1,2,SH,Economy,166169,207872,79.938,1430,73.950,60.941,75.028,91.280,92.245,99.295,82.820,93.766,94.545,99.258
235,2019,7,30,SH,Economy,300251,321241,93.466,2413,86.390,66.009,56.814,75.300,76.498,94.953,63.344,79.022,81.609,95.426
277,2019,9,39,SH,Economy,292282,314057,93.067,2320,86.280,72.038,73.532,89.784,90.614,99.038,77.645,91.808,93.665,99.005
285,2019,10,40,SH,Economy,250316,273119,91.651,1993,83.610,68.762,76.574,90.927,91.631,98.670,78.647,92.178,93.586,98.709


In [42]:
check_dates_df=df[(df['year']==2022)  & (df['cabin']== 'Business') & (df['haul']== 'SH')]

In [45]:
# Create a date range covering the expected time period
start_date = check_dates_df['date_flight_local'].min()
end_date = check_dates_df['date_flight_local'].max()
expected_dates = pd.date_range(start_date, end_date, freq='W')

# Find the missing dates using set operations
missing_dates = expected_dates.difference(check_dates_df['date_flight_local'])

# Print the missing dates
print("Missing dates:")
print(missing_dates)

Missing dates:
DatetimeIndex([], dtype='datetime64[ns]', freq='W-SUN')


In [44]:
adjusted_df.to_csv('outputs/weekly_NPS_5ch_Touchpoints_adjusted_complete', index=False)